In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import re
import pickle
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import en_core_web_md
from xgboost import XGBClassifier

%matplotlib inline
plt.style.use('ggplot')

In [2]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')          
stopWords = list(stopwords.words('english'))   

nlp = en_core_web_md.load()
spacy_text_blob = SpacyTextBlob()
nlp.add_pipe(spacy_text_blob)
def is_stock_up(row):
    return True if row[1] < row[4] else False


def tokenize_paragraph(paragraph):
    raw_sentences = tokenizer.tokenize(paragraph)
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(sentence_to_wordlist(raw_sentence.lower()))
    return sentences
            
        
def remove_stopwords(sentences):
    sent_remove_stopwords = []
    for sent in sentences:
        append_sent = []
        for word in sent:
            if word not in stopWords:
                append_sent.append(word)
        sent_remove_stopwords.append(append_sent)
    return sent_remove_stopwords
        
            
def clean_text(paragraph):
    sentences = tokenize_paragraph(paragraph)
    sent_remove_stopwords = remove_stopwords(sentences)
    words = []
    for sent in sent_remove_stopwords:
        for word in sent:
            words.append(word)
    return words

#check gensim
def get_paragraph_vectors(tokens): 
    num_tokens = len(tokens)
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokens)]
    global doc2vec
    doc2vec = pickle.load(open('doc2vec.sav', 'rb'))
    return [doc2vec.dv[i] for i in range(num_tokens)]

#check gensim
def get_infered_vectors(tokens):
    return model.infer_vector(tokens)
       
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

def reduce_dimensions(vectors):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # reduce using t-SNE
    print('transforming to vectors...')
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=42)
    print('applying tsne...')
    vectors = tsne.fit_transform(vectors)
    print('appending to list...')
    X = [v[0] for v in vectors]
    y = [v[1] for v in vectors]
    return X, y

def get_numpy_array(pandas_series, num_features=300):
    zeroes = np.empty((len(pandas_series),num_features))
    i = 0
    for vectors in pandas_series:
        zeroes[i] = np.array(vectors)
        i+=1
    return zeroes

def get_autoencoder_result(numpy_array):
    encoded_text = encoder.predict(numpy_array)
    return decoder.predict(encoded_text)

def get_vector_difference(initial_vector, decoded_vector):
    return np.absolute(initial_vector.sum() - decoded_vector.sum())

#check gensim
def get_word_vectors(model):
    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.key_to_index:
        vectors.append(model.wv[word])
        labels.append(word)
    return vectors, labels
        
def rearrange_date_string(series):
    dates = series.split('-')
    return dates[2] + '-' + dates[1] + '-' + dates[0]
    
def get_sentiment(body):
    doc = nlp(body)
    return doc._.sentiment.polarity

def get_subjectivity(body):
    doc = nlp(body)
    return doc._.sentiment.subjectivity
    

In [9]:
df = pd.read_csv('cleaned_wsb_sample.csv')

In [10]:
df = df.drop('Unnamed: 0',axis=1)

In [11]:
df.created_utc = pd.DatetimeIndex(data=df.created_utc,tz='Etc/Universal')

TypeError: [datetime.datetime(2020, 3, 25, 4, 11, 50, tzinfo=tzoffset(None, -14400))
 datetime.datetime(2020, 3, 25, 4, 11, 45, tzinfo=tzoffset(None, -14400))
 datetime.datetime(2020, 3, 25, 4, 11, 41, tzinfo=tzoffset(None, -14400))
 ...
 datetime.datetime(2018, 9, 12, 0, 13, 51, tzinfo=tzoffset(None, -14400))
 datetime.datetime(2018, 9, 12, 0, 13, 18, tzinfo=tzoffset(None, -14400))
 datetime.datetime(2018, 9, 12, 0, 13, 4, tzinfo=tzoffset(None, -14400))]

In [12]:
df.created_utc.min()

'2018-02-17 18:25:41-05:00'

In [13]:
df['sentiment'] = df.body.apply(get_sentiment)

In [14]:
df['subjectivity'] = df.body.apply(get_subjectivity)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11180899 entries, 0 to 11180898
Data columns (total 5 columns):
 #   Column        Dtype                        
---  ------        -----                        
 0   body          object                       
 1   created_utc   datetime64[ns, Etc/Universal]
 2   token_length  int64                        
 3   sentiment     float64                      
 4   subjectivity  float64                      
dtypes: datetime64[ns, Etc/Universal](1), float64(2), int64(1), object(1)
memory usage: 426.5+ MB


In [16]:
df.head()

body  \
0  We’re retarded and claim to be often. If you l...   
1                    markets been open for 13 min...   
2  Spy can fuck around all it wants just as long ...   
3                      you lost me at "bulls fucked"   
4  I wonder if the same people who short TSLA but...   

                created_utc  token_length  sentiment  subjectivity  
0 2020-03-25 08:11:50+00:00            17    -0.8000         0.800  
1 2020-03-25 08:11:45+00:00             6     0.0000         0.500  
2 2020-03-25 08:11:41+00:00            20    -0.0625         0.275  
3 2020-03-25 08:11:36+00:00             6    -0.6000         0.700  
4 2020-03-25 08:11:35+00:00            19    -0.1200         0.465

In [18]:
df['created_utc'] = df['created_utc'].apply(lambda timestamp : timestamp.tz_convert('US/Eastern'))

In [40]:
df['date_string'] = df['created_utc'].dt.strftime('%Y-%m-%d')

In [56]:
df['month'] = df['created_utc'].dt.strftime('%B')

In [38]:
df['day_of_week'] = df['created_utc'].dt.strftime('%A')

In [80]:
df.to_csv('cleaned_wsb_sample.csv')

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11180899 entries, 0 to 11180898
Data columns (total 8 columns):
 #   Column        Dtype                     
---  ------        -----                     
 0   body          object                    
 1   created_utc   datetime64[ns, US/Eastern]
 2   token_length  int64                     
 3   sentiment     float64                   
 4   subjectivity  float64                   
 5   date_string   object                    
 6   day_of_week   object                    
 7   month         object                    
dtypes: datetime64[ns, US/Eastern](1), float64(2), int64(1), object(4)
memory usage: 682.4+ MB


In [60]:
count = df.groupby(['date_string','day_of_week','month'])['body'].count()

In [61]:
count

date_string  day_of_week  month   
2018-02-17   Saturday     February      133
2018-02-18   Sunday       February      564
2018-02-19   Monday       February      572
2018-02-20   Tuesday      February      626
2018-02-21   Wednesday    February      536
                                      ...  
2021-02-12   Friday       February    32549
2021-02-13   Saturday     February    12548
2021-02-14   Sunday       February     4206
2021-02-15   Monday       February    11814
2021-02-16   Tuesday      February     5022
Name: body, Length: 1035, dtype: int64

In [68]:
sentiment = df.groupby(['date_string','day_of_week','month'])['sentiment'].mean()
subjectivity = df.groupby(['date_string','day_of_week','month'])['subjectivity'].mean()

In [104]:
sent_df = pd.DataFrame(sentiment)
subj_df = pd.DataFrame(subjectivity)
count_df = pd.DataFrame(count)

In [105]:
sent_df = sent_df.reset_index(level=[0,1,2])
subj_df = subj_df.reset_index(level=[0,1,2,])
count_df = count_df.reset_index(level=[0,1,2])

In [106]:
combined_df = pd.merge(sent_df,subj_df, on='date_string')
combined_df = pd.merge(combined_df,count_df,on='date_string')
combined_df.head()

date_string day_of_week_x   month_x  sentiment day_of_week_y   month_y  \
0  2018-02-17      Saturday  February  -0.008260      Saturday  February   
1  2018-02-18        Sunday  February   0.046003        Sunday  February   
2  2018-02-19        Monday  February   0.059630        Monday  February   
3  2018-02-20       Tuesday  February   0.036545       Tuesday  February   
4  2018-02-21     Wednesday  February   0.042253     Wednesday  February   

   subjectivity day_of_week     month  body  
0      0.383118    Saturday  February   133  
1      0.344501      Sunday  February   564  
2      0.342871      Monday  February   572  
3      0.353234     Tuesday  February   626  
4      0.335483   Wednesday  February   536

In [107]:
combined_df = combined_df.drop(['day_of_week_x','day_of_week_y','month_x','month_y'],axis=1)
combined_df.head()

date_string  sentiment  subjectivity day_of_week     month  body
0  2018-02-17  -0.008260      0.383118    Saturday  February   133
1  2018-02-18   0.046003      0.344501      Sunday  February   564
2  2018-02-19   0.059630      0.342871      Monday  February   572
3  2018-02-20   0.036545      0.353234     Tuesday  February   626
4  2018-02-21   0.042253      0.335483   Wednesday  February   536

In [110]:
combined_df = combined_df.rename(mapper={'body':'comment_count'},axis=1)
combined_df.head()

date_string  sentiment  subjectivity day_of_week     month  comment_count
0  2018-02-17  -0.008260      0.383118    Saturday  February            133
1  2018-02-18   0.046003      0.344501      Sunday  February            564
2  2018-02-19   0.059630      0.342871      Monday  February            572
3  2018-02-20   0.036545      0.353234     Tuesday  February            626
4  2018-02-21   0.042253      0.335483   Wednesday  February            536

In [113]:
combined_df['day_of_week'] = pd.Categorical(combined_df['day_of_week'])
combined_df['month'] = pd.Categorical(combined_df['month'])
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   date_string    1035 non-null   object  
 1   sentiment      1035 non-null   float64 
 2   subjectivity   1035 non-null   float64 
 3   day_of_week    1035 non-null   category
 4   month          1035 non-null   category
 5   comment_count  1035 non-null   int64   
dtypes: category(2), float64(2), int64(1), object(1)
memory usage: 75.5+ KB


In [114]:
combined_df.to_csv('training_data.csv')

In [120]:
combined_df

date_string  sentiment  subjectivity day_of_week     month  comment_count
0     2018-02-17  -0.008260      0.383118    Saturday  February            133
1     2018-02-18   0.046003      0.344501      Sunday  February            564
2     2018-02-19   0.059630      0.342871      Monday  February            572
3     2018-02-20   0.036545      0.353234     Tuesday  February            626
4     2018-02-21   0.042253      0.335483   Wednesday  February            536
...          ...        ...           ...         ...       ...            ...
1030  2021-02-12   0.052434      0.352311      Friday  February          32549
1031  2021-02-13   0.055929      0.368345    Saturday  February          12548
1032  2021-02-14   0.059450      0.370586      Sunday  February           4206
1033  2021-02-15   0.052697      0.359491      Monday  February          11814
1034  2021-02-16   0.056642      0.349215     Tuesday  February           5022

[1035 rows x 6 columns]

In [95]:
combined_df = pd.read_csv('training_data.csv')
combined_df = combined_df.drop('Unnamed: 0',axis=1)
combined_df['day_of_week'] = pd.Categorical(combined_df['day_of_week'])
combined_df['month'] = pd.Categorical(combined_df['month'])
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   date_string    1035 non-null   object  
 1   sentiment      1035 non-null   float64 
 2   subjectivity   1035 non-null   float64 
 3   day_of_week    1035 non-null   category
 4   month          1035 non-null   category
 5   comment_count  1035 non-null   int64   
dtypes: category(2), float64(2), int64(1), object(1)
memory usage: 35.2+ KB


In [114]:
stocks = ['SP500.csv','GME.csv','BB.csv','AMC.csv','NOK.csv','PLTR.csv','TSLA.csv']
df_stocks = []
for stock in stocks:
    df_stocks.append(pd.read_csv(stock))
    
filtered_stocks = []
for frame in df_stocks:
    frame.Date = pd.DatetimeIndex(data=frame.Date)
    filtered_stocks.append(frame[frame.Date >= datetime(2018, 2, 15)]) 
df_stocks = []
for frame in filtered_stocks:
    frame.Date = pd.DatetimeIndex(data=frame.Date,tz='US/Eastern')
    df_stocks.append(frame)
    
for frame in df_stocks:
    frame['up_today'] = frame.apply(is_stock_up,axis=1)

In [115]:
def clean_open(open_price,close_price):
    open_price = list(open_price)
    close_price = list(close_price)
    cleanned_prices = [open_price[0]]
    cleanned_prices.extend(close_price[:-1])
    return cleanned_prices

time_erase = df_stocks[1].iloc[1,0]
def rotate_df_series(movements):
    first_half = movements[1:]
    second_half = movements[0]
    first_half.append(second_half)
    return first_half

def remove_first_row(df):
    row = df[df.Date<=time_erase].index
    return df.drop(row)

In [116]:
for frame in df_stocks:
    frame['Open'] = clean_open(frame.Open,frame.Close)

In [117]:
i = 0
for frame in df_stocks:
    frame['pct_change'] = -(frame['Open'] - frame['Close']) / frame['Open']

In [118]:
cleanned_frames = []
for frame in df_stocks:
    movements = list(frame.up_today)
    tomorrow = rotate_df_series(movements)
    frame['up_tomorrow'] = tomorrow
    frame = remove_first_row(frame)
    cleanned_frames.append(frame)
df_stocks = []
for frame in cleanned_frames:
    frame = remove_first_row(frame)
    df_stocks.append(frame)

In [119]:
for frame in df_stocks:
    frame['date_string'] = frame['Date'].dt.strftime('%Y-%m-%d')
    frame['day_of_week'] = frame['Date'].dt.strftime('%A')
    frame['month'] = frame['Date'].dt.strftime('%B')
    

In [120]:
for frame in df_stocks:
    frame['pct_volatility'] = (frame['High'] - frame['Low']) / frame['Low']

In [121]:
for frame in df_stocks:
    frame.drop(['Date','Adj Close'],axis=1, inplace=True)

In [131]:
test_df = test.merge(test1)

In [133]:
testing_frames = []
for frame in df_stocks:
    testing_frames.append(frame.merge(combined_df))

In [143]:
i = 0
for frame in testing_frames:
    
    frame['day_of_week'] = pd.Categorical(frame['day_of_week'])
    frame['month'] = pd.Categorical(frame['month'])
    testing_frames[i] = pd.get_dummies(frame, columns=['day_of_week','month'], drop_first=True)
    i += 1
    
testing_frames[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748 entries, 0 to 747
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Open                   748 non-null    float64
 1   High                   748 non-null    float64
 2   Low                    748 non-null    float64
 3   Close                  748 non-null    float64
 4   Volume                 748 non-null    float64
 5   up_today               748 non-null    bool   
 6   pct_change             748 non-null    float64
 7   up_tomorrow            748 non-null    bool   
 8   date_string            748 non-null    object 
 9   pct_volatility         748 non-null    float64
 10  sentiment              748 non-null    float64
 11  subjectivity           748 non-null    float64
 12  comment_count          748 non-null    int64  
 13  day_of_week_Monday     748 non-null    uint8  
 14  day_of_week_Thursday   748 non-null    uint8  
 15  day_of

In [165]:
class StockDataClassification():   
    
    def __init__(self, df):
        self.model = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.score = None
        self.X = df.drop(['up_tomorrow','date_string'],axis=1)
        self.y = df['up_tomorrow']
        
    def get_train_test_split(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.2, random_state = 42)
        
    def train_model(self, model):
        self.model = model
        self.model = self.model.fit(self.X_train, self.y_train)
        
    def get_model_results(self):
        self.score = self.model.score(self.X_test,self.y_test)
        return self.score
    
    def get_model_improvement(self):
        return self.score - self.y.mean()

In [163]:
class StockDataPctChange(StockDataClassification):
    
    def __init__(self, df):   
        self.model = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.X = df.drop(['up_tomorrow','date_string','pct_change'],axis=1)
        self.y = df['pct_change']
    

In [ ]:
['SP500.csv','GME.csv','BB.csv','AMC.csv','NOK.csv','PLTR.csv','TSLA.csv']

In [185]:
gme = StockDataClassification(testing_frames[5])

In [186]:
gme.get_train_test_split()
gme.train_model(XGBClassifier())
gme.get_model_results()

[15:09:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ryan\anaconda3\envs\capstone\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.6111111111111112

In [187]:
gme.get_model_improvement()

0.2666666666666667

In [194]:
stocks = ['SP500','GME','BB','AMC','NOK','PLTR','TSLA']
i = 0
results_dict = {}
for frame in testing_frames:
    stock_classification = StockDataClassification(frame)
    stock_classification.get_train_test_split()
    stock_classification.train_model(XGBClassifier(use_label_encoder=False))
    results = stock_classification.get_model_results()
    improvement = stock_classification.get_model_improvement()
    results_dict[stocks[i]] = [results,improvement]
    i += 1

[15:59:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:59:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:59:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:59:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [197]:
i = 0
for key in results_dict:
    print('''For stock {stock}, the model accuracy is {results}, 
The improvement over the baseline model is {improvement}'''.format(
        stock=stocks[i],
        results=results_dict[stocks[i]][0],
        improvement=results_dict[stocks[i]][1]))
    i += 1

For stock SP500, the model accuracy is 0.5466666666666666, 
The improvement over the baseline model is -0.001461675579322641
For stock GME, the model accuracy is 0.5466666666666666, 
The improvement over the baseline model is 0.07474153297682706
For stock BB, the model accuracy is 0.52, 
The improvement over the baseline model is 0.07213903743315508
For stock AMC, the model accuracy is 0.5266666666666666, 
The improvement over the baseline model is 0.09217468805704093
For stock NOK, the model accuracy is 0.47333333333333333, 
The improvement over the baseline model is 0.057557932263814615
For stock PLTR, the model accuracy is 0.6111111111111112, 
The improvement over the baseline model is 0.2666666666666667
For stock TSLA, the model accuracy is 0.4375, 
The improvement over the baseline model is -0.060407949790795
